In [5]:
import tensorflow as tf
import numpy as np
import glob
import os
from PIL import Image # Image後面會用到 #圖片轉換成二進位

ModuleNotFoundError: No module named 'tensorflow'

In [6]:
cwd = 'C:/Users/Java/Desktop/Project/Project_Image/'
classes = {'渡月橋','清水寺','龍安寺','金閣寺','平等院鳳凰堂','三十三間堂','千本鳥居','京都火車站','二條城'}
writer= tf.python_io.TFRecordWriter("kyoto_train.tfrecords") #要生成的文件

NameError: name 'tf' is not defined

In [7]:
for index,name in enumerate(classes):
    class_path=cwd+name+'/'  # 圖片資料夾
    for img_name in os.listdir(class_path): 
        img_path=class_path+img_name # 每張圖片的位置

        img=Image.open(img_path)
        img= img.resize((128,128))
        img_raw=img.tobytes()#圖片轉換成二進位
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        })) 
        #example对象对label和image数据进行封装
        #tf.python_io.TFRecordWriter 写入到TFRecords文件中
        writer.write(example.SerializeToString())  #序列化为字符串

NameError: name 'os' is not defined

In [8]:
writer.close()

NameError: name 'writer' is not defined

In [9]:
def read_and_decode(filename): # 讀取kyoto_train.tfrecords
    filename_queue = tf.train.string_input_producer([filename])#生成一个queue队列

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })#将image数据和label取出来

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [128, 128, 3])  #reshape为128*128的3通道图片
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5 #在流中抛出img张量
    label = tf.cast(features['label'], tf.int32) #在流中抛出label张量
    return img, label

In [10]:
read_and_decode("kyoto_train.tfrecords")

NameError: name 'tf' is not defined

In [11]:
filename_queue = tf.train.string_input_producer(["kyoto_train.tfrecords"]) #读入流中
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)   #返回文件名和文件
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                   })  #取出包含image和label的feature对象
image = tf.decode_raw(features['img_raw'], tf.uint8)
image = tf.reshape(image, [128, 128, 3])
label = tf.cast(features['label'], tf.int32)
with tf.Session() as sess: #开始一个会话
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    for i in range(30000):
        try:
            example, l = sess.run([image,label])#在Session中取出image和label
            img=Image.fromarray(example, 'RGB')#这里Image是之前提到的
            img.save(cwd+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
            print(example, l)
        except:
            pass
    coord.request_stop()
    coord.join(threads)

NameError: name 'tf' is not defined

In [ ]:
import pandas as pd

In [ ]:
imagedata = read_and_decode("kyoto_train.tfrecords")